# Introduction
This notebook includes toy examples to demonstrate how to tune User Defined Functions with `flaml.tune`.

FLAML requires `Python>=3.7`. To run this notebook example, please install flaml with the `notebook` option:
```bash
pip install flaml[notebook]
```

In [ ]:
%pip install flaml[notebook]
# from v0.6.6, catboost is made an optional dependency to build conda package.
# to install catboost without installing the notebook option, you can run:
# %pip install flaml[catboost]

# Basic tuning procedure
## 1. A basic tuning example

In [18]:
'''Set a search space'''
from flaml import tune
config_search_space = {
    "x": tune.lograndint(lower=1, upper=100000),
    "y": tune.randint(lower=1, upper=100000)
}  

In [19]:
'''Write a evaluation function'''
import time
def evaluate_config(config: dict):
    """evaluate a hyperparameter configuration"""
    score = (config["x"] - 85000) ** 2 - config["x"] / config["y"]
    # usually the evaluation takes an non-neglible cost
    # and the cost could be related to certain hyperparameters
    # here we simulate this cost by calling the time.sleep() function
    # here we assume the cost is proportional to x
    faked_evaluation_cost = config["x"] / 100000
    time.sleep(faked_evaluation_cost)
    # we can return a single float as a score on the input config:
    # return score
    # or, we can return a dictionary that maps metric name to metric value:
    return {"score": score, "evaluation_cost": faked_evaluation_cost, "constraint_metric": config["x"] * config["y"]}

In [20]:
''''Performs tuning'''
# require: pip install flaml[blendsearch]
analysis = tune.run(
    evaluate_config,  # the function to evaluate a config
    config=config_search_space,  # the search space defined
    metric="score",
    mode="min",  # the optimization mode, "min" or "max"
    num_samples=-1,  # the maximal number of configs to try, -1 means infinite
    time_budget_s=10,  # the time budget in seconds
)

You passed a `space` parameter to OptunaSearch that contained unresolved search space definitions. OptunaSearch should however be instantiated with fully configured search spaces only. To use Ray Tune's automatic search space conversion, pass the space definition as part of the `config` argument to `tune.run()` instead.
[I 2022-07-07 04:02:16,168] A new study created in memory with name: optuna
[flaml.tune.tune: 07-07 04:02:16] {506} INFO - trial 1 config: {'x': 3, 'y': 13184}
[flaml.tune.tune: 07-07 04:02:16] {506} INFO - trial 2 config: {'x': 6134, 'y': 2076}
[flaml.tune.tune: 07-07 04:02:16] {506} INFO - trial 3 config: {'x': 1143, 'y': 74880}
[flaml.tune.tune: 07-07 04:02:16] {506} INFO - trial 4 config: {'x': 5539, 'y': 1}
[flaml.tune.tune: 07-07 04:02:16] {506} INFO - trial 5 config: {'x': 6793, 'y': 16190}
[flaml.tune.tune: 07-07 04:02:16] {506} INFO - trial 6 config: {'x': 220, 'y': 22480}
[flaml.tune.tune: 07-07 04:02:16] {506} INFO - trial 7 config: {'x': 6, 'y': 76053}
[flam

In [21]:
'''Investigate results'''
print(analysis.best_result)

{'score': 138344643.26761267, 'evaluation_cost': 0.73238, 'constraint_metric': 7323726762, 'training_iteration': 0, 'config': {'x': 73238, 'y': 99999}, 'config/x': 73238, 'config/y': 99999, 'experiment_tag': 'exp', 'time_total_s': 0.7350201606750488}


## Hierarchical search space 
Hierarchical search space is supported.

In [22]:
'''Set a hierarchical search space'''
from flaml import tune
gbtree_hp_space = {
        "booster": "gbtree",
        "n_estimators": tune.lograndint(lower=4, upper=64),
        "max_leaves": tune.lograndint(lower=4, upper=64),
        "learning_rate": tune.loguniform(lower=1 / 1024, upper=1.0),
    }
gblinear_hp_space = {
    "booster": "gblinear",
    "lambda": tune.uniform(0, 1),
    "alpha": tune.loguniform(0.0001, 1),
}

full_space = {
    "xgb_config": tune.choice([gbtree_hp_space, gblinear_hp_space]),
}

In [23]:
'''Write a evaluation function'''
import xgboost as xgb
def xgb_obj(X_train, X_test, y_train, y_test, config):
    config = config["xgb_config"]
    params = config2params(config)
    dtrain = xgb.DMatrix(X_train, label=y_train)
    booster_type = config.get("booster")

    if booster_type == "gblinear":
        model = xgb.train(
            params,
            dtrain,
        )
    else:
        _n_estimators = params.pop("n_estimators")
        model = xgb.train(params, dtrain, _n_estimators)

    # get validation loss
    from sklearn.metrics import r2_score

    dtest = xgb.DMatrix(X_test)
    y_test_predict = model.predict(dtest)
    test_loss = 1.0 - r2_score(y_test, y_test_predict)
    return {"loss": test_loss}

def config2params(config: dict) -> dict:
    params = config.copy()
    max_depth = params["max_depth"] = params.get("max_depth", 0)
    if max_depth == 0:
        params["grow_policy"] = params.get("grow_policy", "lossguide")
        params["tree_method"] = params.get("tree_method", "hist")
    # params["booster"] = params.get("booster", "gbtree")
    params["use_label_encoder"] = params.get("use_label_encoder", False)
    if "n_jobs" in config:
        params["nthread"] = params.pop("n_jobs")
    return params

In [24]:
'''Tune xgb_obj with configs from the hierarchical search space'''
from flaml.data import load_openml_dataset
from functools import partial

X_train, X_test, y_train, y_test = load_openml_dataset(
    dataset_id=537, data_dir="./"
)
analysis = tune.run(
    partial(xgb_obj, X_train, X_test, y_train, y_test),
    config=full_space,
    metric="loss",
    mode="min",
    num_samples=5,
)
print("analysis", analysis.results)

[I 2022-07-07 04:02:27,568] A new study created in memory with name: optuna
[flaml.tune.tune: 07-07 04:02:27] {506} INFO - trial 1 config: {'xgb_config': {'booster': 'gblinear', 'lambda': 0.6472660813321921, 'alpha': 0.0028264214081400014}}
/home/qxw5138/miniconda3/envs/myflaml/lib/python3.8/site-packages/xgboost/data.py:192: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index
[flaml.tune.tune: 07-07 04:02:27] {506} INFO - trial 2 config: {'xgb_config': {'n_estimators': 22, 'max_leaves': 31, 'learning_rate': 0.0309282737630552, 'booster': 'gbtree'}}


load dataset from ./openml_ds537.pkl
Dataset name: houses
X_train.shape: (15480, 8), y_train.shape: (15480,);
X_test.shape: (5160, 8), y_test.shape: (5160,)
[04:02:27] WARNING: ../src/learner.cc:541: 
Parameters: { grow_policy, max_depth, tree_method, use_label_encoder } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[04:02:27] WARNING: ../src/learner.cc:541: 
Parameters: { use_label_encoder } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.




/home/qxw5138/miniconda3/envs/myflaml/lib/python3.8/site-packages/xgboost/data.py:192: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index
[flaml.tune.tune: 07-07 04:02:27] {506} INFO - trial 3 config: {'xgb_config': {'n_estimators': 32, 'max_leaves': 6, 'learning_rate': 0.0018014797394283806, 'booster': 'gbtree'}}
/home/qxw5138/miniconda3/envs/myflaml/lib/python3.8/site-packages/xgboost/data.py:192: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index
[flaml.tune.tune: 07-07 04:02:27] {506} INFO - trial 4 config: {'xgb_config': {'n_estimators': 21, 'max_leaves': 21, 'learning_rate': 0.06308266770250766, 'booster': 'gbtree'}}
/home/qxw5138/miniconda3/envs/myflaml/lib/python3.8/site-packages/xgboost/dat

[04:02:27] WARNING: ../src/learner.cc:541: 
Parameters: { use_label_encoder } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[04:02:27] WARNING: ../src/learner.cc:541: 
Parameters: { use_label_encoder } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[04:02:27] WARNING: ../src/learner.cc:541: 
Parameters: { use_label_encoder } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find abov

# Advanced Tuning Options

## 1. Constraints on the tuning

1. A user can specify constraints on the configurations to be satisfied via the argument `config_constraints`.

In [25]:
from flaml import tune
def area(config):
    return config["width"] * config["length"]

cube_search_space = {
    "width": tune.lograndint(lower=1, upper=1000),
    "height": tune.randint(lower=1, upper=1000),
    "length": tune.randint(lower=1, upper=1000),
}

def cube_volume(config: dict):
    """evaluate a hyperparameter configuration"""
    score = config["width"] * config["height"] * config["length"]
    return {"volume": score}

analysis = tune.run(evaluation_function=cube_volume,
         mode="min",
         metric="volume",
         config=cube_search_space,
         config_constraints=[(area, "<=", 1000)],
         num_samples=5,
        )
print(analysis.best_result)

You passed a `space` parameter to OptunaSearch that contained unresolved search space definitions. OptunaSearch should however be instantiated with fully configured search spaces only. To use Ray Tune's automatic search space conversion, pass the space definition as part of the `config` argument to `tune.run()` instead.
[I 2022-07-07 04:02:28,659] A new study created in memory with name: optuna
[flaml.tune.tune: 07-07 04:02:28] {506} INFO - trial 1 config: {'width': 1, 'height': 132, 'length': 647}
[flaml.tune.tune: 07-07 04:02:28] {506} INFO - trial 2 config: {'width': 2, 'height': 760, 'length': 169}
[flaml.tune.tune: 07-07 04:02:28] {506} INFO - trial 3 config: {'width': 1, 'height': 685, 'length': 953}
[flaml.tune.tune: 07-07 04:02:28] {506} INFO - trial 4 config: {'width': 1, 'height': 512, 'length': 812}
[flaml.tune.tune: 07-07 04:02:28] {506} INFO - trial 5 config: {'width': 1, 'height': 373, 'length': 674}


{'volume': 85404, 'training_iteration': 0, 'config': {'width': 1, 'height': 132, 'length': 647}, 'config/width': 1, 'config/height': 132, 'config/length': 647, 'experiment_tag': 'exp', 'time_total_s': 0.0014650821685791016}


2.  You can also specify a list of metric constraints to be satisfied via the argument `metric_constraints`.

In [28]:
from flaml import tune
tune.run(evaluation_function=evaluate_config,
         mode="min",
         metric="score",
         config=config_search_space,
         metric_constraints=[("evaluation_cost", "<=", 0.1)],
         num_samples=5,)

You passed a `space` parameter to OptunaSearch that contained unresolved search space definitions. OptunaSearch should however be instantiated with fully configured search spaces only. To use Ray Tune's automatic search space conversion, pass the space definition as part of the `config` argument to `tune.run()` instead.
[I 2022-07-07 04:03:36,539] A new study created in memory with name: optuna
[flaml.tune.tune: 07-07 04:03:36] {506} INFO - trial 1 config: {'x': 3, 'y': 13184}
[flaml.tune.tune: 07-07 04:03:36] {506} INFO - trial 2 config: {'x': 6134, 'y': 2076}
[flaml.tune.tune: 07-07 04:03:36] {506} INFO - trial 3 config: {'x': 1143, 'y': 74880}
[flaml.tune.tune: 07-07 04:03:36] {506} INFO - trial 4 config: {'x': 5539, 'y': 1}
[flaml.tune.tune: 07-07 04:03:36] {506} INFO - trial 5 config: {'x': 6793, 'y': 16190}


## Parallel tuning

In [ ]:
%pip install flaml[ray]

In [29]:
# require: pip install flaml[ray]
analysis = tune.run(
    evaluate_config,  # the function to evaluate a config
    config=config_search_space,  # the search space defined
    metric="score",
    mode="min",  # the optimization mode, "min" or "max"
    num_samples=-1,  # the maximal number of configs to try, -1 means infinite
    time_budget_s=10,  # the time budget in seconds
    use_ray=True,
    resources_per_trial={"cpu": 2}  # limit resources allocated per trial
)
print(analysis.best_trial.last_result)  # the best trial's result
print(analysis.best_config)  # the best config

You passed a `space` parameter to OptunaSearch that contained unresolved search space definitions. OptunaSearch should however be instantiated with fully configured search spaces only. To use Ray Tune's automatic search space conversion, pass the space definition as part of the `config` argument to `tune.run()` instead.
[I 2022-07-07 04:04:55,820] A new study created in memory with name: optuna
2022-07-07 04:04:59,245	WARNING function_runner.py:603 -- Function checkpointing is disabled. This may result in unexpected behavior when using checkpointing features or certain schedulers. To enable, set the train function arguments to be `func(config, checkpoint_dir=None)`.
2022-07-07 04:04:59,774	WARNING tune.py:668 -- Tune detects GPUs, but no trials are using GPUs. To enable trials to use GPUs, set tune.run(resources_per_trial={'gpu': 1}...) which allows Tune to expose 1 GPU to each trial. You can also override `Trainable.default_resource_request` if using the Trainable API.


Trial evaluate_config_a49d1df2 reported score=7224490009.00 with parameters={'x': 3, 'y': 13184}.
Trial evaluate_config_a49d1df2 completed. Last result: score=7224490008.999772,evaluation_cost=3e-05,constraint_metric=39552
Trial evaluate_config_a5a79c2d reported score=7031996448.98 with parameters={'x': 1143, 'y': 74880}.
Trial evaluate_config_a5a79c2d completed. Last result: score=7031996448.9847355,evaluation_cost=0.01143,constraint_metric=85587840
Trial evaluate_config_a5a79c2f reported score=7223980036.00 with parameters={'x': 6, 'y': 76053}.
Trial evaluate_config_a5a79c2f completed. Last result: score=7223980035.999921,evaluation_cost=6e-05,constraint_metric=456318
Trial evaluate_config_a5a79c31 reported score=6864453903.98 with parameters={'x': 2148, 'y': 95339}.
Trial evaluate_config_a5a79c31 completed. Last result: score=6864453903.977469,evaluation_cost=0.02148,constraint_metric=204788172
Trial evaluate_config_a5a79c34 reported score=6303883608.92 with parameters={'x': 5603, '

Trial evaluate_config_a694ab54 completed. Last result: score=24940034.4007436,evaluation_cost=0.80006,constraint_metric=4002460162
Trial evaluate_config_a694ab59 completed. Last result: score=623301153.6395235,evaluation_cost=0.60034,constraint_metric=1526844722
Trial evaluate_config_a5a79c4e completed. Last result: score=7224830000.999984,evaluation_cost=1e-05,constraint_metric=62629
Trial evaluate_config_a694ab65 reported score=4785734040.09 with parameters={'x': 15821, 'y': 17465}.
Trial evaluate_config_a694ab51 reported score=195859938.84 with parameters={'x': 98995, 'y': 1149}.
Trial evaluate_config_a694ab65 completed. Last result: score=4785734040.094131,evaluation_cost=0.15821,constraint_metric=276313765
Trial evaluate_config_a694ab66 reported score=2899284022.48 with parameters={'x': 31155, 'y': 12371}.
Trial evaluate_config_a694ab63 reported score=1847022494.33 with parameters={'x': 42023, 'y': 1212}.
Trial evaluate_config_a694ab51 completed. Last result: score=195859938.84247

2022-07-07 04:05:10,036	INFO stopper.py:363 -- Reached timeout of 10 seconds. Stopping all trials.


Trial evaluate_config_a8ed9145 reported score=118222121.74 with parameters={'x': 95873, 'y': 13212}.
Trial evaluate_config_aa2213ad completed. Last result: score=1505983245.7208774,evaluation_cost=0.46193,constraint_metric=650720791
Trial evaluate_config_aa2213a4 reported score=39375618.55 with parameters={'x': 91275, 'y': 14152}.
Trial evaluate_config_a8ed9145 completed. Last result: score=118222121.74349077,evaluation_cost=0.95873,constraint_metric=1266674076


Trial name,status,loc,x,y,iter,total time (s),score,evaluation_cost,constraint_metric
evaluate_config_a49d1df2,TERMINATED,130.203.136.143:4004202,3,13184,1,0.000576019,7.22449e+09,3e-05,39552
evaluate_config_a5a79c2c,TERMINATED,130.203.136.143:4004235,6134,2076,1,0.0619102,6.21985e+09,0.06134,12734184
evaluate_config_a5a79c2d,TERMINATED,130.203.136.143:4004202,1143,74880,1,0.0118678,7.032e+09,0.01143,85587840
evaluate_config_a5a79c2e,TERMINATED,130.203.136.143:4004241,220,22480,1,0.00351667,7.18765e+09,0.0022,4945600
evaluate_config_a5a79c2f,TERMINATED,130.203.136.143:4004202,6,76053,1,0.000468969,7.22398e+09,6e-05,456318
evaluate_config_a5a79c30,TERMINATED,130.203.136.143:4004246,4,8834,1,0.000578165,7.22432e+09,4e-05,35336
evaluate_config_a5a79c31,TERMINATED,130.203.136.143:4004202,2148,95339,1,0.0219069,6.86445e+09,0.02148,204788172
evaluate_config_a5a79c32,TERMINATED,130.203.136.143:4004253,1032,60766,1,0.0107296,7.05063e+09,0.01032,62710512
evaluate_config_a5a79c33,TERMINATED,130.203.136.143:4004257,1266,88994,1,0.0130882,7.01138e+09,0.01266,112666404
evaluate_config_a5a79c34,TERMINATED,130.203.136.143:4004202,5603,71825,1,0.0565019,6.30388e+09,0.05603,402435475


2022-07-07 04:05:11,081	INFO tune.py:747 -- Total run time: 11.84 seconds (10.54 seconds for the tuning loop).


{'score': 25229.14119804401, 'evaluation_cost': 0.84841, 'constraint_metric': 138799876, 'time_this_iter_s': 0.8488931655883789, 'done': True, 'timesteps_total': None, 'episodes_total': None, 'training_iteration': 1, 'trial_id': 'a694ab52', 'experiment_id': 'fd9619caaad1438884ac27a5c6171e14', 'date': '2022-07-07_04-05-04', 'timestamp': 1657191904, 'time_total_s': 0.8488931655883789, 'pid': 4004241, 'hostname': 'i4-l-qxw5138-01.ad.psu.edu', 'node_ip': '130.203.136.143', 'config': {'x': 84841, 'y': 1636}, 'time_since_restore': 0.8488931655883789, 'timesteps_since_restore': 0, 'iterations_since_restore': 1, 'warmup_time': 0.00529789924621582, 'experiment_tag': '48_x=84841,y=1636'}
{'x': 84841, 'y': 1636}


## Warm start

In [31]:
from flaml import tune

config_search_space = {
    "a": tune.uniform(lower=0, upper=0.99),
    "b": tune.uniform(lower=0, upper=3),
}

def simple_obj(config):
    return config["a"] + config["b"]

points_to_evaluate = [
    {"b": .99, "a": 3},
    {"b": .99, "a": 2},
    {"b": .80, "a": 3},
    {"b": .80, "a": 2},
]
evaluated_rewards = [3.99, 2.99]

analysis = tune.run(
    simple_obj,
    config=config_search_space,
    mode="max",
    points_to_evaluate=points_to_evaluate,
    evaluated_rewards=evaluated_rewards,
    num_samples=10,
)

2022-07-07 04:09:43,543	WARNING optuna.py:297 -- You passed a `space` parameter to OptunaSearch that contained unresolved search space definitions. OptunaSearch should however be instantiated with fully configured search spaces only. To use Ray Tune's automatic search space conversion, pass the space definition as part of the `config` argument to `tune.run()` instead.
[I 2022-07-07 04:09:43,547] A new study created in memory with name: optuna
2022-07-07 04:09:43,549	WARNING optuna.py:297 -- You passed a `space` parameter to OptunaSearch that contained unresolved search space definitions. OptunaSearch should however be instantiated with fully configured search spaces only. To use Ray Tune's automatic search space conversion, pass the space definition as part of the `config` argument to `tune.run()` instead.
[I 2022-07-07 04:09:43,552] A new study created in memory with name: optuna
[flaml.tune.tune: 07-07 04:09:43] {506} INFO - trial 1 config: {'b': 0.8, 'a': 3.0}
[flaml.tune.tune: 07-0

## Trial scheduling

###  An authentic scheduler implemented in FLAML (`scheduler='flaml'`).

In [40]:
search_space = {
        "n_estimators": tune.lograndint(lower=4, upper=32768),
        "max_leaves": tune.lograndint(lower=4, upper=32768),
        "learning_rate": tune.loguniform(lower=1 / 1024, upper=1.0),
    }

In [41]:
'''Set a evaluation function with resource dimension'''
def obj_from_resource_attr(resource_attr, X_train, X_test, y_train, y_test, config):
    from lightgbm import LGBMClassifier
    from sklearn.metrics import accuracy_score

    # in this example sample size is our resource dimension
    resource = int(config[resource_attr])
    sampled_X_train = X_train.iloc[:resource]
    sampled_y_train = y_train[:resource]

    # construct a LGBM model from the config
    # note that you need to first remove the resource_attr field
    # from the config as it is not part of the original search space
    model_config = config.copy()
    del model_config[resource_attr]
    model = LGBMClassifier(**model_config)

    model.fit(sampled_X_train, sampled_y_train)
    y_test_predict = model.predict(X_test)
    test_loss = 1.0 - accuracy_score(y_test, y_test_predict)
    return {"loss": test_loss}

In [42]:
from flaml import tune
from functools import partial
from flaml.data import load_openml_task
    
X_train, X_test, y_train, y_test = load_openml_task(task_id=7592, data_dir="")
max_resource = len(y_train)
resource_attr = "sample_size"
min_resource = 1000
analysis = tune.run(
    partial(
        obj_from_resource_attr, resource_attr, X_train, X_test, y_train, y_test
    ),
    config=search_space,
    metric="loss",
    mode="min",
    resource_attr=resource_attr,
    scheduler="flaml",
    max_resource=max_resource,
    min_resource=min_resource,
    reduction_factor=2,
    time_budget_s=10,
    num_samples=-1,
)
print("best result w/ flaml scheduler (in 10s): ", analysis.best_result)

2022-07-07 04:19:14,463	WARNING optuna.py:297 -- You passed a `space` parameter to OptunaSearch that contained unresolved search space definitions. OptunaSearch should however be instantiated with fully configured search spaces only. To use Ray Tune's automatic search space conversion, pass the space definition as part of the `config` argument to `tune.run()` instead.
[I 2022-07-07 04:19:14,465] A new study created in memory with name: optuna
[flaml.tune.tune: 07-07 04:19:14] {506} INFO - trial 1 config: {'n_estimators': 9, 'max_leaves': 1364, 'learning_rate': 0.012074374674294664, 'sample_size': 1000}
[flaml.tune.tune: 07-07 04:19:14] {506} INFO - trial 2 config: {'n_estimators': 4048, 'max_leaves': 4, 'learning_rate': 0.07891713267442702, 'sample_size': 1000}
[flaml.tune.tune: 07-07 04:19:14] {506} INFO - trial 3 config: {'n_estimators': 3295, 'max_leaves': 334, 'learning_rate': 0.004638797085780012, 'sample_size': 1000}
[flaml.tune.tune: 07-07 04:19:14] {506} INFO - trial 4 config: 

load dataset from openml_task7592.pkl
X_train.shape: (43957, 14), y_train.shape: (43957,),
X_test.shape: (4885, 14), y_test.shape: (4885,)
[LightGBM] [Warning] num_leaves is set=31, max_leaves=1364 will be ignored. Current value: num_leaves=31
[LightGBM] [Warning] num_leaves is set=31, max_leaves=4 will be ignored. Current value: num_leaves=31
[LightGBM] [Warning] num_leaves is set=31, max_leaves=334 will be ignored. Current value: num_leaves=31
[LightGBM] [Warning] num_leaves is set=31, max_leaves=3668 will be ignored. Current value: num_leaves=31
[LightGBM] [Warning] num_leaves is set=31, max_leaves=1845 will be ignored. Current value: num_leaves=31


[flaml.tune.tune: 07-07 04:19:14] {506} INFO - trial 6 config: {'n_estimators': 4, 'max_leaves': 379, 'learning_rate': 0.2728556109672425, 'sample_size': 1000}
[flaml.tune.tune: 07-07 04:19:14] {506} INFO - trial 7 config: {'n_estimators': 948, 'max_leaves': 2573, 'learning_rate': 0.0073847289359894605, 'sample_size': 1000}
[flaml.tune.tune: 07-07 04:19:14] {506} INFO - trial 8 config: {'n_estimators': 15449, 'max_leaves': 2409, 'learning_rate': 0.04196829547317673, 'sample_size': 1000}
[flaml.tune.tune: 07-07 04:19:14] {506} INFO - trial 9 config: {'n_estimators': 13, 'max_leaves': 106, 'learning_rate': 0.10448271169801053, 'sample_size': 1000}
[flaml.tune.tune: 07-07 04:19:14] {506} INFO - trial 10 config: {'n_estimators': 199, 'max_leaves': 185, 'learning_rate': 0.07069097177435173, 'sample_size': 1000}


[LightGBM] [Warning] num_leaves is set=31, max_leaves=379 will be ignored. Current value: num_leaves=31
[LightGBM] [Warning] num_leaves is set=31, max_leaves=2573 will be ignored. Current value: num_leaves=31
[LightGBM] [Warning] num_leaves is set=31, max_leaves=2409 will be ignored. Current value: num_leaves=31
[LightGBM] [Warning] num_leaves is set=31, max_leaves=106 will be ignored. Current value: num_leaves=31
[LightGBM] [Warning] num_leaves is set=31, max_leaves=185 will be ignored. Current value: num_leaves=31


[flaml.tune.tune: 07-07 04:19:14] {506} INFO - trial 11 config: {'n_estimators': 382, 'max_leaves': 1340, 'learning_rate': 0.06295171682621144, 'sample_size': 1000}
[flaml.tune.tune: 07-07 04:19:14] {506} INFO - trial 12 config: {'n_estimators': 5520, 'max_leaves': 413, 'learning_rate': 0.5308914484951052, 'sample_size': 1000}
[flaml.tune.tune: 07-07 04:19:14] {506} INFO - trial 13 config: {'n_estimators': 3503, 'max_leaves': 7, 'learning_rate': 0.0010918443163764437, 'sample_size': 1000}
[flaml.tune.tune: 07-07 04:19:15] {506} INFO - trial 14 config: {'n_estimators': 54, 'max_leaves': 30204, 'learning_rate': 0.004360679238440181, 'sample_size': 1000}
[flaml.tune.tune: 07-07 04:19:15] {506} INFO - trial 15 config: {'n_estimators': 49, 'max_leaves': 23677, 'learning_rate': 0.0010641529214071718, 'sample_size': 1000}
[flaml.tune.tune: 07-07 04:19:15] {506} INFO - trial 16 config: {'n_estimators': 1534, 'max_leaves': 37, 'learning_rate': 0.006371183957388993, 'sample_size': 1000}


...

[LightGBM] [Warning] num_leaves is set=31, max_leaves=4 will be ignored. Current value: num_leaves=31
[LightGBM] [Warning] num_leaves is set=31, max_leaves=3066 will be ignored. Current value: num_leaves=31


[flaml.tune.tune: 07-07 04:19:24] {506} INFO - trial 181 config: {'n_estimators': 14379, 'max_leaves': 21532, 'learning_rate': 0.11249756313060533, 'sample_size': 1000}
[flaml.tune.tune: 07-07 04:19:24] {506} INFO - trial 182 config: {'n_estimators': 43, 'max_leaves': 6487, 'learning_rate': 0.17296950883370182, 'sample_size': 1000}
[flaml.tune.tune: 07-07 04:19:24] {506} INFO - trial 183 config: {'n_estimators': 22, 'max_leaves': 7280, 'learning_rate': 0.258852421157622, 'sample_size': 1000}
[flaml.tune.tune: 07-07 04:19:24] {506} INFO - trial 184 config: {'n_estimators': 45, 'max_leaves': 3571, 'learning_rate': 0.08732512289838663, 'sample_size': 1000}
[flaml.tune.tune: 07-07 04:19:24] {506} INFO - trial 185 config: {'n_estimators': 30, 'max_leaves': 14583, 'learning_rate': 0.002891946623707599, 'sample_size': 1000}
[flaml.tune.tune: 07-07 04:19:24] {506} INFO - trial 186 config: {'n_estimators': 25, 'max_leaves': 3464, 'learning_rate': 0.002358433026191249, 'sample_size': 1000}


[LightGBM] [Warning] num_leaves is set=31, max_leaves=21532 will be ignored. Current value: num_leaves=31
[LightGBM] [Warning] num_leaves is set=31, max_leaves=6487 will be ignored. Current value: num_leaves=31
[LightGBM] [Warning] num_leaves is set=31, max_leaves=7280 will be ignored. Current value: num_leaves=31
[LightGBM] [Warning] num_leaves is set=31, max_leaves=3571 will be ignored. Current value: num_leaves=31
[LightGBM] [Warning] num_leaves is set=31, max_leaves=14583 will be ignored. Current value: num_leaves=31
[LightGBM] [Warning] num_leaves is set=31, max_leaves=3464 will be ignored. Current value: num_leaves=31


[flaml.tune.tune: 07-07 04:19:24] {506} INFO - trial 187 config: {'n_estimators': 66, 'max_leaves': 10033, 'learning_rate': 0.001881199853774848, 'sample_size': 1000}
[flaml.tune.tune: 07-07 04:19:24] {506} INFO - trial 188 config: {'n_estimators': 45, 'max_leaves': 2290, 'learning_rate': 0.0032255721973679058, 'sample_size': 1000}
[flaml.tune.tune: 07-07 04:19:24] {506} INFO - trial 189 config: {'n_estimators': 21382, 'max_leaves': 2208, 'learning_rate': 0.04681316555326558, 'sample_size': 1000}


[LightGBM] [Warning] num_leaves is set=31, max_leaves=10033 will be ignored. Current value: num_leaves=31
[LightGBM] [Warning] num_leaves is set=31, max_leaves=2290 will be ignored. Current value: num_leaves=31
[LightGBM] [Warning] num_leaves is set=31, max_leaves=2208 will be ignored. Current value: num_leaves=31
best result w/ flaml scheduler (in 10s):  {'loss': 0.2393039918116684, 'training_iteration': 0, 'config': {'n_estimators': 9, 'max_leaves': 1364, 'learning_rate': 0.012074374674294664, 'sample_size': 1000}, 'config/n_estimators': 9, 'config/max_leaves': 1364, 'config/learning_rate': 0.012074374674294664, 'config/sample_size': 1000, 'experiment_tag': 'exp', 'time_total_s': 0.022912979125976562}


###  ASHA scheduler (`scheduler='asha'`) or a custom scheduler of the  [`TrialScheduler`](https://docs.ray.io/en/latest/tune/api_docs/schedulers.html#tune-schedulers) class from `ray.tune`.

In [43]:
def obj_w_intermediate_report(
        resource_attr,
        X_train,
        X_test,
        y_train,
        y_test,
        min_resource,
        max_resource,
        config,
    ):
    from lightgbm import LGBMClassifier
    from sklearn.metrics import accuracy_score

    # a customized schedule to perform the evaluation
    eval_schedule = [res for res in range(min_resource, max_resource, 5000)] + [
        max_resource
    ]
    for resource in eval_schedule:
        sampled_X_train = X_train.iloc[:resource]
        sampled_y_train = y_train[:resource]

        # construct a LGBM model from the config
        model = LGBMClassifier(**config)

        model.fit(sampled_X_train, sampled_y_train)
        y_test_predict = model.predict(X_test)
        test_loss = 1.0 - accuracy_score(y_test, y_test_predict)
        # need to report the resource attribute used and the corresponding intermediate results
        try:
            tune.report(sample_size=resource, loss=test_loss)
        except StopIteration:
            return

In [44]:
X_train, X_test, y_train, y_test = load_openml_task(task_id=7592, data_dir="")
resource_attr = "sample_size"
min_resource = 1000
max_resource = len(y_train)
analysis = tune.run(
    partial(
        obj_w_intermediate_report,
        resource_attr,
        X_train,
        X_test,
        y_train,
        y_test,
        min_resource,
        max_resource,
    ),
    config=search_space,
    metric="loss",
    mode="min",
    resource_attr=resource_attr,
    scheduler="asha",
    max_resource=max_resource,
    min_resource=min_resource,
    reduction_factor=2,
    time_budget_s=10,
    num_samples=-1,
)
print("best result w/ asha scheduler (in 10s): ", analysis.best_result)

2022-07-07 04:19:32,319	WARNING optuna.py:297 -- You passed a `space` parameter to OptunaSearch that contained unresolved search space definitions. OptunaSearch should however be instantiated with fully configured search spaces only. To use Ray Tune's automatic search space conversion, pass the space definition as part of the `config` argument to `tune.run()` instead.
[I 2022-07-07 04:19:32,321] A new study created in memory with name: optuna
[flaml.tune.tune: 07-07 04:19:32] {506} INFO - trial 1 config: {'n_estimators': 9, 'max_leaves': 1364, 'learning_rate': 0.012074374674294664}


load dataset from openml_task7592.pkl
X_train.shape: (43957, 14), y_train.shape: (43957,),
X_test.shape: (4885, 14), y_test.shape: (4885,)
[LightGBM] [Warning] num_leaves is set=31, max_leaves=1364 will be ignored. Current value: num_leaves=31
[LightGBM] [Warning] num_leaves is set=31, max_leaves=1364 will be ignored. Current value: num_leaves=31
[LightGBM] [Warning] num_leaves is set=31, max_leaves=1364 will be ignored. Current value: num_leaves=31
[LightGBM] [Warning] num_leaves is set=31, max_leaves=1364 will be ignored. Current value: num_leaves=31
[LightGBM] [Warning] num_leaves is set=31, max_leaves=1364 will be ignored. Current value: num_leaves=31
[LightGBM] [Warning] num_leaves is set=31, max_leaves=1364 will be ignored. Current value: num_leaves=31
[LightGBM] [Warning] num_leaves is set=31, max_leaves=1364 will be ignored. Current value: num_leaves=31
[LightGBM] [Warning] num_leaves is set=31, max_leaves=1364 will be ignored. Current value: num_leaves=31
[LightGBM] [Warning] 

[flaml.tune.tune: 07-07 04:19:33] {506} INFO - trial 2 config: {'n_estimators': 4048, 'max_leaves': 4, 'learning_rate': 0.07891713267442702}


[LightGBM] [Warning] num_leaves is set=31, max_leaves=1364 will be ignored. Current value: num_leaves=31
[LightGBM] [Warning] num_leaves is set=31, max_leaves=4 will be ignored. Current value: num_leaves=31
[LightGBM] [Warning] num_leaves is set=31, max_leaves=4 will be ignored. Current value: num_leaves=31
[LightGBM] [Warning] num_leaves is set=31, max_leaves=4 will be ignored. Current value: num_leaves=31
[LightGBM] [Warning] num_leaves is set=31, max_leaves=4 will be ignored. Current value: num_leaves=31
[LightGBM] [Warning] num_leaves is set=31, max_leaves=4 will be ignored. Current value: num_leaves=31
[LightGBM] [Warning] num_leaves is set=31, max_leaves=4 will be ignored. Current value: num_leaves=31
[LightGBM] [Warning] num_leaves is set=31, max_leaves=4 will be ignored. Current value: num_leaves=31
[LightGBM] [Warning] num_leaves is set=31, max_leaves=4 will be ignored. Current value: num_leaves=31
[LightGBM] [Warning] num_leaves is set=31, max_leaves=4 will be ignored. Curren